In [22]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import seaborn as sns
%matplotlib inline

## Get the NFL play-by-play data from 2016-2019 from http://www.nflsavant.com/about.php (Download as csv)

In [24]:
for year in range(2016, 2020):
    year = str(year)
    # reads the original pbp csv and add a time remaining column
    data = pd.read_csv("pbp-" + year + ".csv", encoding='iso-8859-1')
    data['Remaining'] = data['Quarter']*(4-2)*15*60 + data['Minute']*60 + data['Second']
    data.to_csv('pdb-' + year + 'csv')

    corr_dict = {'ne':'nwe', 'no':'nor', 'lac':'sdg', 'sd':'sdg', 'la':'ram', 'sf':'sfo', 'tb':'tam', 'kc':'kan', 'gb':'gnb', 'ten':'oti','hou':'htx','bal':'rav','oak':'rai','ari':'crd','ind':'clt'}

    # convert date and team name to a possible game id
    def get_id(date, team):
        game_id = str(date)[0:8]+'0'+team.lower()
        for name in corr_dict.keys():
            if name in game_id:
                game_id = game_id.replace(name, corr_dict[name])
                break
        return game_id

    # using the possible game id to find the score table of that game 
    def get_score(gameid):
        score_list = []
        url = 'https://www.pro-football-reference.com/boxscores/' + gameid + '.htm'
        print(url)
        try:
            response = requests.get(url)
            results_page = BeautifulSoup(response.content,'lxml')
            scoring = results_page.find("table", {"id": 'scoring'})
            home_team = scoring.thead.find("tr").find("th", {"data-stat": 'home_team_score'}).text
            vis_team = scoring.thead.find("tr").find("th", {"data-stat": 'vis_team_score'}).text
            scoring_tr = scoring.tbody.find_all("tr")
            curr_quarter = 1
            for tr in scoring_tr:
                score_dict = dict()
                score_dict['GameId'] = gameid
                score_dict['HomeTeam'] = home_team
                score_dict['VisTeam'] = vis_team
                quarter = tr.find("th", {"data-stat": 'quarter'}).text
                if quarter == '':
                    quarter = curr_quarter
                else:
                    curr_quarter = quarter
                score_dict['Quarter'] = quarter
                time = tr.find("td", {"data-stat": 'time'}).text
                time_split = time.split(':')
                remaining = (4-int(quarter))*15*60 + int(time_split[0])*60 + int(time_split[1])
                score_dict['Time'] = time
                score_dict['Remaining'] = remaining
                home_score = tr.find("td", {"data-stat": 'home_team_score'}).text
                score_dict['HomeScore'] = home_score
                vis_score = tr.find("td", {"data-stat": 'vis_team_score'}).text
                score_dict['VisScore'] = vis_score
                score_list.append(score_dict)
            return score_list
        except Exception as e: 
            return score_list

    # reads the pbp csv, eliminate data with NaN offense team or defense team, and drop duplicates
    data = pd.read_csv("pbp-" + year + ".csv", encoding='iso-8859-1')
    data_game = data[(pd.isnull(data['OffenseTeam'])==False) & (pd.isnull(data['DefenseTeam'])==False)]
    data_game = data_game.drop_duplicates(subset='GameId')

    # compute all the possible game ids
    games_list = []
    for index, row in data_game.iterrows():
        game_id_1 = get_id(row[0], row[6])
        game_id_2 = get_id(row[0], row[7])
        games_list.append(game_id_1)
        games_list.append(game_id_2)

    # get the score tables for all the games in this year and output it into a csv
    all_list = []
    for game in games_list:
        all_list.extend(get_score(game))
    # all_list
    df = pd.DataFrame(all_list)
    df.to_csv('score_' + year + '.csv')

    # read the score csv
    score = pd.read_csv('score_' + year + '.csv')

    # select only pass, rush, scramble plays in the pbp csv
    pbp = pd.read_csv("pbp-" + year + ".csv", encoding='iso-8859-1')
    pbp = pbp[(pbp['PlayType']=='PASS') | (pbp['PlayType']=='RUSH') | (pbp['PlayType']=='SCRAMBLE')]
    pbp = pbp[~((pd.isnull(pbp['OffenseTeam'])) | (pd.isnull(pbp['DefenseTeam'])))]
    pbp['PlayType'] = np.where(pbp['PlayType']=='SCRAMBLE','PASS',pbp['PlayType'])

    # for these plays, calculate the score diff in the perspective of the offense team and create a column for it
    diff_list = []
    for index, row in pbp.iterrows():
        offense_team = row[6]
        remaining = int(row[5])
        game_id_1 = get_id(row[0], row[6])
        game_id_2 = get_id(row[0], row[7])

        sub_df = score[score['GameId'] == game_id_1]
        sub_df = sub_df.append(score[score['GameId'] == game_id_2])
        sub_df = sub_df.reset_index()
        diff = -1
        for i in range(len(sub_df['Remaining'])):
            if sub_df['Remaining'][i] < remaining:
                if i == 0:
                    diff = 0
                time_index = i-1
                break
            elif sub_df['Remaining'][i] == remaining:
                time_index = i
            elif i == len(sub_df['Remaining'])-1 and sub_df['Remaining'][i] > remaining:
                time_index = i
        try:
            if diff == -1:
                if offense_team == sub_df['HomeTeam'][time_index]:
                    diff = int(sub_df['HomeScore'][time_index]) - int(sub_df['VisScore'][time_index])
                else:
                    diff = int(sub_df['VisScore'][time_index]) - int(sub_df['HomeScore'][time_index])
        except Exception as e: 
            print('Failure', e)
            print(index)
            print(row[6], game_id_1, row[7], game_id_2)
            print(sub_df)
            break
        diff_list.append(diff)
    pbp['PointDiff'] = diff_list
    pbp.to_csv('pbp_' + year + '_adj.csv')

https://www.pro-football-reference.com/boxscores/201609110htx.htm
https://www.pro-football-reference.com/boxscores/201609110chi.htm
https://www.pro-football-reference.com/boxscores/201609110nor.htm
https://www.pro-football-reference.com/boxscores/201609110rai.htm
https://www.pro-football-reference.com/boxscores/201609110cin.htm
https://www.pro-football-reference.com/boxscores/201609110nyj.htm
https://www.pro-football-reference.com/boxscores/201609110cle.htm
https://www.pro-football-reference.com/boxscores/201609110phi.htm
https://www.pro-football-reference.com/boxscores/201609110min.htm
https://www.pro-football-reference.com/boxscores/201609110oti.htm
https://www.pro-football-reference.com/boxscores/201611270rai.htm
https://www.pro-football-reference.com/boxscores/201611270car.htm
https://www.pro-football-reference.com/boxscores/201611270nwe.htm
https://www.pro-football-reference.com/boxscores/201611270nyj.htm
https://www.pro-football-reference.com/boxscores/201609110sea.htm
https://ww

https://www.pro-football-reference.com/boxscores/201610020ram.htm
https://www.pro-football-reference.com/boxscores/201610020nor.htm
https://www.pro-football-reference.com/boxscores/201610020sdg.htm
https://www.pro-football-reference.com/boxscores/201610020dal.htm
https://www.pro-football-reference.com/boxscores/201610020sfo.htm
https://www.pro-football-reference.com/boxscores/201610170nyj.htm
https://www.pro-football-reference.com/boxscores/201610170crd.htm
https://www.pro-football-reference.com/boxscores/201610020pit.htm
https://www.pro-football-reference.com/boxscores/201610020kan.htm
https://www.pro-football-reference.com/boxscores/201612040atl.htm
https://www.pro-football-reference.com/boxscores/201612040kan.htm
https://www.pro-football-reference.com/boxscores/201612040rav.htm
https://www.pro-football-reference.com/boxscores/201612040mia.htm
https://www.pro-football-reference.com/boxscores/201610030min.htm
https://www.pro-football-reference.com/boxscores/201610030nyg.htm
https://ww

https://www.pro-football-reference.com/boxscores/201610300sdg.htm
https://www.pro-football-reference.com/boxscores/201610300den.htm
https://www.pro-football-reference.com/boxscores/201612110tam.htm
https://www.pro-football-reference.com/boxscores/201612110nor.htm
https://www.pro-football-reference.com/boxscores/201611130nyj.htm
https://www.pro-football-reference.com/boxscores/201611130ram.htm
https://www.pro-football-reference.com/boxscores/201611270oti.htm
https://www.pro-football-reference.com/boxscores/201611270chi.htm
https://www.pro-football-reference.com/boxscores/201610300crd.htm
https://www.pro-football-reference.com/boxscores/201610300car.htm
https://www.pro-football-reference.com/boxscores/201612040sea.htm
https://www.pro-football-reference.com/boxscores/201612040car.htm
https://www.pro-football-reference.com/boxscores/201610300phi.htm
https://www.pro-football-reference.com/boxscores/201610300dal.htm
https://www.pro-football-reference.com/boxscores/201610310chi.htm
https://ww

https://www.pro-football-reference.com/boxscores/201611270cin.htm
https://www.pro-football-reference.com/boxscores/201611270jax.htm
https://www.pro-football-reference.com/boxscores/201611270buf.htm
https://www.pro-football-reference.com/boxscores/201611270nyg.htm
https://www.pro-football-reference.com/boxscores/201611270cle.htm
https://www.pro-football-reference.com/boxscores/201611270sdg.htm
https://www.pro-football-reference.com/boxscores/201611270htx.htm
https://www.pro-football-reference.com/boxscores/201611270sfo.htm
https://www.pro-football-reference.com/boxscores/201611270mia.htm
https://www.pro-football-reference.com/boxscores/201611270nor.htm
https://www.pro-football-reference.com/boxscores/201611270ram.htm
https://www.pro-football-reference.com/boxscores/201611270tam.htm
https://www.pro-football-reference.com/boxscores/201611270sea.htm
https://www.pro-football-reference.com/boxscores/201611270den.htm
https://www.pro-football-reference.com/boxscores/201611270kan.htm
https://ww

https://www.pro-football-reference.com/boxscores/201701010kan.htm
https://www.pro-football-reference.com/boxscores/201701010sdg.htm
https://www.pro-football-reference.com/boxscores/201701010was.htm
https://www.pro-football-reference.com/boxscores/201701010nyg.htm
https://www.pro-football-reference.com/boxscores/201701010sfo.htm
https://www.pro-football-reference.com/boxscores/201701010sea.htm
https://www.pro-football-reference.com/boxscores/201609080car.htm
https://www.pro-football-reference.com/boxscores/201609080den.htm
https://www.pro-football-reference.com/boxscores/201609110tam.htm
https://www.pro-football-reference.com/boxscores/201609110atl.htm
https://www.pro-football-reference.com/boxscores/201609110gnb.htm
https://www.pro-football-reference.com/boxscores/201609110jax.htm
https://www.pro-football-reference.com/boxscores/201709070kan.htm
https://www.pro-football-reference.com/boxscores/201709070nwe.htm
https://www.pro-football-reference.com/boxscores/201709100pit.htm
https://ww

...

https://www.pro-football-reference.com/boxscores/201810080was.htm
https://www.pro-football-reference.com/boxscores/201810070htx.htm
https://www.pro-football-reference.com/boxscores/201810070dal.htm
https://www.pro-football-reference.com/boxscores/201810070sea.htm
https://www.pro-football-reference.com/boxscores/201810070ram.htm
https://www.pro-football-reference.com/boxscores/201810070crd.htm
https://www.pro-football-reference.com/boxscores/201810070sfo.htm
https://www.pro-football-reference.com/boxscores/201810070min.htm
https://www.pro-football-reference.com/boxscores/201810070phi.htm
https://www.pro-football-reference.com/boxscores/201810070atl.htm
https://www.pro-football-reference.com/boxscores/201810070pit.htm
https://www.pro-football-reference.com/boxscores/201810070den.htm
https://www.pro-football-reference.com/boxscores/201810070nyj.htm
https://www.pro-football-reference.com/boxscores/201810070jax.htm
https://www.pro-football-reference.com/boxscores/201810070kan.htm
https://ww

https://www.pro-football-reference.com/boxscores/201809090gnb.htm
https://www.pro-football-reference.com/boxscores/201809090chi.htm
https://www.pro-football-reference.com/boxscores/201809090den.htm
https://www.pro-football-reference.com/boxscores/201809090sea.htm
https://www.pro-football-reference.com/boxscores/201809090car.htm
https://www.pro-football-reference.com/boxscores/201809090dal.htm
https://www.pro-football-reference.com/boxscores/201809090sdg.htm
https://www.pro-football-reference.com/boxscores/201809090kan.htm
https://www.pro-football-reference.com/boxscores/201809100rai.htm
https://www.pro-football-reference.com/boxscores/201809100ram.htm
https://www.pro-football-reference.com/boxscores/201809090jax.htm
https://www.pro-football-reference.com/boxscores/201809090nyg.htm
https://www.pro-football-reference.com/boxscores/201809090nor.htm
https://www.pro-football-reference.com/boxscores/201809090tam.htm
https://www.pro-football-reference.com/boxscores/201809090htx.htm
https://ww

https://www.pro-football-reference.com/boxscores/201911030clt.htm
https://www.pro-football-reference.com/boxscores/201911030car.htm
https://www.pro-football-reference.com/boxscores/201911030oti.htm
https://www.pro-football-reference.com/boxscores/201910270cle.htm
https://www.pro-football-reference.com/boxscores/201910270nwe.htm
https://www.pro-football-reference.com/boxscores/201910270tam.htm
https://www.pro-football-reference.com/boxscores/201910270oti.htm
https://www.pro-football-reference.com/boxscores/201911030kan.htm
https://www.pro-football-reference.com/boxscores/201911030min.htm
https://www.pro-football-reference.com/boxscores/201911030mia.htm
https://www.pro-football-reference.com/boxscores/201911030nyj.htm
https://www.pro-football-reference.com/boxscores/201910280pit.htm
https://www.pro-football-reference.com/boxscores/201910280mia.htm
https://www.pro-football-reference.com/boxscores/201911030buf.htm
https://www.pro-football-reference.com/boxscores/201911030was.htm
https://ww

https://www.pro-football-reference.com/boxscores/201910060dal.htm
https://www.pro-football-reference.com/boxscores/201910060gnb.htm
https://www.pro-football-reference.com/boxscores/201910060htx.htm
https://www.pro-football-reference.com/boxscores/201910060atl.htm
https://www.pro-football-reference.com/boxscores/201909290nor.htm
https://www.pro-football-reference.com/boxscores/201909290dal.htm
https://www.pro-football-reference.com/boxscores/201909290jax.htm
https://www.pro-football-reference.com/boxscores/201909290den.htm
https://www.pro-football-reference.com/boxscores/201909290chi.htm
https://www.pro-football-reference.com/boxscores/201909290min.htm
https://www.pro-football-reference.com/boxscores/201909300pit.htm
https://www.pro-football-reference.com/boxscores/201909300cin.htm
https://www.pro-football-reference.com/boxscores/201909290tam.htm
https://www.pro-football-reference.com/boxscores/201909290ram.htm
https://www.pro-football-reference.com/boxscores/201909290crd.htm
https://ww

https://www.pro-football-reference.com/boxscores/201909080mia.htm
https://www.pro-football-reference.com/boxscores/201909080car.htm
https://www.pro-football-reference.com/boxscores/201909080ram.htm
https://www.pro-football-reference.com/boxscores/201909050gnb.htm
https://www.pro-football-reference.com/boxscores/201909050chi.htm


Get the team offense and defense stats data from https://www.pro-football-reference.com/years/2018/ and https://www.pro-football-reference.com/years/2018/opp.htm (Download the data as csv files)

In [25]:
# convert the abbreviation team name of the pbp file to the abbreviation team name used by the pro-football-reference website
def pbp_to_pro(abbr):
    corr_dict = {'ne':'nwe', 'no':'nor', 'lac':'sdg', 'sd':'sdg', 'la':'ram', 'sf':'sfo', 'tb':'tam', 'kc':'kan', 'gb':'gnb', 'ten':'oti','hou':'htx','bal':'rav','oak':'rai','ari':'crd','ind':'clt'}
    if abbr in corr_dict.keys():
        return corr_dict[abbr]
    return abbr

# convert the abbreviation team name of the train file to the abbreviation team name used by the pro-football-reference website
def train_to_pro(abbr):    
    train_name_dict = {'ARZ': 'crd', 'ARI': 'crd', 'ATL': 'atl', 'BLT': 'rav', 'BAL': 'rav', 'BUF': 'buf', 'CAR': 'car', 'CHI': 'chi', 'CIN': 'cin', 'CLV': 'cle', 'CLE': 'cle', 'DAL': 'dal', 'DEN': 'den', 'DET': 'det', 'GB': 'gnb', 'HST': 'htx', 'HOU': 'htx', 'IND': 'clt', 'JAX': 'jax', 'KC': 'kan', 'LAC': 'sdg', 'LA': 'ram', 'MIA': 'mia', 'MIN': 'min', 'NE': 'nwe', 'NO': 'nor', 'NYG': 'nyg', 'NYJ': 'nyj', 'OAK': 'rai', 'PHI': 'phi', 'PIT': 'pit', 'SF': 'sfo', 'SEA': 'sea', 'TB': 'tam', 'TEN': 'oti', 'WAS': 'was'}
    return train_name_dict[abbr]

# create a dict of full team name and abbreviation used by the pro-football-reference website
def get_team_name():
    team_name_dict = dict()
    url = 'https://www.pro-football-reference.com/teams/'
    response = requests.get(url)
    result_page = BeautifulSoup(response.content,'lxml')
    team_table = result_page.find("table", {"id": 'teams_active'}).tbody
    team_rows = team_table.find_all('tr')
    for row in team_rows:
        team = row.find('th', {'class': 'left'})
        if team is not None:
            team_name_dict[team.text] = team.find('a').get('href')[7:10]
    team_name_dict['San Diego Chargers'] = 'sdg'
    return team_name_dict

# input a full team name, return the abbreviation used by the pro-football-reference website 
def get_team_abbr(team_list):
    team_name_dict = get_team_name()
    result = []
    for team in team_list:
        result.append(team_name_dict[team])
    return result

# calculate the team rank given a list of teams and their stats in one category
def get_team_rank(stats):
    sorted_list = []
    index = 0
    diff = stats.max() - stats.min()
    for i in stats:
        percent = (i-stats.min())/diff
        if percent >= 0.75:
            sorted_list.append('A')
        elif percent >= 0.5:
            sorted_list.append('B')
        elif percent >= 0.25:
            sorted_list.append('C')
        else:
            sorted_list.append('D')
    return sorted_list

# reverse the rank(since sometimes higher number doesn't mean better)
def reverse_rank(stats):
    r_dict = {'A':'D', 'B':'C', 'C':'B', 'D':'A'}
    return r_dict[stats]

# Given a dataframe, calculate all team ranks on every columns of categories
def get_rank_df(df):
    df_copy = df.copy()
    columns = list(df_copy)
    for col in columns[3:-1]:
        df_copy[col] = get_team_rank(df[col])
    return df_copy

# adding all the useful team ranks into a new df
def add_team_stats(df):
    off_pass_cmp_list = []
    off_pass_att_list = []
    off_pass_cmprate_list = []
    off_pass_td_list = []
    off_intrate_list  = []
    off_sackrate_list = []
    off_pass_yds_list = []
    off_pass_yds_game_list = []
    off_pass_yds_atm_list = []
    off_qbr_list = []
    off_rush_att_list = []
    off_rush_yds_list = []
    off_rush_yds_game_list = []
    off_rush_yds_atm_list = []
    off_rush_td_list = []
    
    def_pass_cmp_list = []
    def_pass_att_list = []
    def_pass_cmprate_list = []
    def_pass_td_list = []
    def_intrate_list  = []
    def_sackrate_list = []
    def_pass_yds_list = []
    def_pass_yds_game_list = []
    def_pass_yds_atm_list = []
    def_rush_att_list = []
    def_rush_yds_list = []
    def_rush_yds_game_list = []
    def_rush_yds_atm_list = []
    def_rush_td_list = []
    
    for index, row in df.iterrows():
        off_team = pbp_to_pro(row[7].lower())
        def_team = pbp_to_pro(row[8].lower())
        off_pass_cmp_list.append(team_pass_rank['Cmp'][off_team])
        off_pass_att_list.append(team_pass_rank['Att'][off_team])
        off_pass_cmprate_list.append(team_pass_rank['Cmp%'][off_team])
        off_pass_td_list.append(team_pass_rank['TD'][off_team])
        off_intrate_list.append(reverse_rank(team_pass_rank['Int%'][off_team]))
        off_sackrate_list.append(reverse_rank(team_pass_rank['Sk%'][off_team]))
        off_pass_yds_list.append(team_pass_rank['Yds'][off_team])
        off_pass_yds_game_list.append(team_pass_rank['Y/G'][off_team])
        off_pass_yds_atm_list.append(team_pass_rank['Y/A'][off_team])
        off_qbr_list.append(team_pass_rank['Rate'][off_team])
        off_rush_att_list.append(team_rush_rank['Att'][off_team])
        off_rush_yds_list.append(team_rush_rank['Yds'][off_team])
        off_rush_yds_game_list.append(team_rush_rank['Y/G'][off_team])
        off_rush_yds_atm_list.append(team_rush_rank['Y/A'][off_team])
        off_rush_td_list.append(team_rush_rank['TD'][off_team])
        
        def_pass_cmp_list.append(reverse_rank(team_passdef_rank['Cmp'][def_team]))
        def_pass_att_list.append(reverse_rank(team_passdef_rank['Att'][def_team]))
        def_pass_cmprate_list.append(reverse_rank(team_passdef_rank['Cmp%'][def_team]))
        def_pass_td_list.append(reverse_rank(team_passdef_rank['TD'][def_team]))
        def_intrate_list.append(team_passdef_rank['Int%'][def_team])
        def_sackrate_list.append(team_passdef_rank['Sk%'][def_team])
        def_pass_yds_list.append(reverse_rank(team_passdef_rank['Yds'][def_team]))
        def_pass_yds_game_list.append(reverse_rank(team_passdef_rank['Y/G'][def_team]))
        def_pass_yds_atm_list.append(reverse_rank(team_passdef_rank['Y/A'][def_team]))
        def_rush_att_list.append(reverse_rank(team_rushdef_rank['Att'][def_team]))
        def_rush_yds_list.append(reverse_rank(team_rushdef_rank['Yds'][def_team]))
        def_rush_yds_game_list.append(reverse_rank(team_rushdef_rank['Y/G'][def_team]))
        def_rush_yds_atm_list.append(reverse_rank(team_rushdef_rank['Y/A'][def_team]))
        def_rush_td_list.append(reverse_rank(team_rushdef_rank['TD'][def_team]))
        
    df['Off Pass Cmp'] = off_pass_cmp_list
    df['Off Pass Att'] = off_pass_att_list
    df['Off Pass Cmp%'] = off_pass_cmprate_list
    df['Off Pass TD'] = off_pass_td_list
    df['Off Pass Int%'] = off_intrate_list
    df['Off Pass Sk%'] = off_sackrate_list
    df['Off Pass Yds'] = off_pass_yds_list
    df['Off Pass Y/G'] = off_pass_yds_game_list
    df['Off Pass Y/A'] = off_pass_yds_atm_list
    df['Off Pass QBR'] = off_qbr_list
    df['Off Rush Att'] = off_rush_att_list
    df['Off Rush Yds'] = off_rush_yds_list
    df['Off Rush Y/G'] = off_rush_yds_game_list
    df['Off Rush Y/A'] = off_rush_yds_atm_list
    df['Off Rush TD'] = off_rush_td_list

    df['Def Pass Cmp'] = def_pass_cmp_list
    df['Def Pass Att'] = def_pass_att_list
    df['Def Pass Cmp%'] = def_pass_cmprate_list
    df['Def Pass TD'] = def_pass_td_list
    df['Def Pass Int%'] = def_intrate_list
    df['Def Pass Sk%'] = def_sackrate_list
    df['Def Pass Yds'] = def_pass_yds_list
    df['Def Pass Y/G'] = def_pass_yds_game_list
    df['Def Pass Y/A'] = def_pass_yds_atm_list
    df['Def Rush Att'] = def_rush_att_list
    df['Def Rush Yds'] = def_rush_yds_list
    df['Def Rush Y/G'] = def_rush_yds_game_list
    df['Def Rush Y/A'] = def_rush_yds_atm_list
    df['Def Rush TD'] = def_rush_td_list
    
    return df

# convert grade to numbers
def grade_dict(grade):
    grade_d = {'A':4, 'B':3, 'C':2, 'D':1}
    return grade_d[grade]

# get the overall ranks based on the sum of stats
def get_overall_rank(stats, num):
    max_score = num*4
    min_score = num*1
    diff = max_score - min_score
    percent = ((stats)-min_score)/diff
    if percent >= 0.75:
        return 'A'
    elif percent >= 0.5:
        return 'B'
    elif percent >= 0.25:
        return 'C'
    else:
        return 'D'

# add the overall team rank in 4 categories into the df
def add_overall_rank(df):   
    off_pass_overall_list = []
    off_rush_overall_list = []
    def_pass_overall_list = []
    def_rush_overall_list = []
    
    for index, row in df.iterrows(): 
        off_pass_overall_list.append(get_overall_rank(grade_dict(row[49])+grade_dict(row[50])\
                                 +grade_dict(row[51])+grade_dict(row[52])+grade_dict(row[53])\
                                 +grade_dict(row[54])+grade_dict(row[57]),7))
        off_rush_overall_list.append(get_overall_rank(grade_dict(row[58])+grade_dict(row[59])+grade_dict(row[62]),3))
        def_pass_overall_list.append(get_overall_rank(grade_dict(row[64])+grade_dict(row[65])\
                                 +grade_dict(row[66])+grade_dict(row[67])+grade_dict(row[68])\
                                 +grade_dict(row[69]),6))
        def_rush_overall_list.append(get_overall_rank(grade_dict(row[72])+grade_dict(row[73])+grade_dict(row[76]),3))
        
    df['Off_Pass_Overall'] = off_pass_overall_list
    df['Off_Rush_Overall'] = off_rush_overall_list
    df['Def_Pass_Overall'] = def_pass_overall_list
    df['Def_Rush_Overall'] = def_rush_overall_list
        
    return df

In [26]:
for year in range(2016, 2020):
    year = str(year)
    # read the team passing offense data
    team_pass = pd.read_csv(year + '_pass.csv')
    team_pass = team_pass[:-3]

    # read the team passing defense data
    team_passdef = pd.read_csv(year + '_passdef.csv')
    team_passdef = team_passdef[:-3]

    # read the team rushing offense data
    team_rush = pd.read_csv(year + '_rush.csv')
    team_rush = team_rush[:-3]

    # read the team rushing defense data
    team_rushdef = pd.read_csv(year + '_rushdef.csv')
    team_rushdef = team_rushdef[:-3]

    # Create columns of team name abbreviations
    team_pass['TmAbbr'] = get_team_abbr(team_pass['Tm'])
    team_rush['TmAbbr'] = get_team_abbr(team_rush['Tm'])
    team_passdef['TmAbbr'] = get_team_abbr(team_passdef['Tm'])
    team_rushdef['TmAbbr'] = get_team_abbr(team_rushdef['Tm'])

    # calculate all the team ranks on all kinds of passing/rushing offense/defense categories
    team_pass_rank = get_rank_df(team_pass)
    team_rush_rank = get_rank_df(team_rush)
    team_pass_rank.set_index('TmAbbr', inplace=True)
    team_rush_rank.set_index('TmAbbr', inplace=True)
    team_passdef_rank = get_rank_df(team_passdef)
    team_rushdef_rank = get_rank_df(team_rushdef)
    team_passdef_rank.set_index('TmAbbr', inplace=True)
    team_rushdef_rank.set_index('TmAbbr', inplace=True)

    # This is used for creating tables for demostration of the program
    # team_pass_rank.to_csv('2018pass.csv')
    # team_rush_rank.to_csv('2018rush.csv')
    # team_passdef_rank.to_csv('2018passdef.csv')
    # team_rushdef_rank.to_csv('2018rushdef.csv')

    # read the pbp play by play file and calculate team ranks of the offense and defense team for every play
    pbp_adj = pd.read_csv('pbp_' + year + '_adj.csv')
    pbp_team = add_overall_rank(add_team_stats(pbp_adj))
    pbp_team.to_csv('pbp_' + year + '_team.csv')

# combine all csvs
all_filenames = ['pbp_2016_team.csv','pbp_2017_team.csv','pbp_2018_team.csv','pbp_2019_team.csv']
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames])
combined_csv.to_csv( "pbp_2016_2019.csv", index=False)

<b>In the belowed preprocessing part, we do the following works:</b>
- Numeric features cleaning
- Text features cleaning
- Generate numeric values for team rankings

In [2]:
df = pd.read_csv(r'C:\Users\zhong\Downloads\pbp_2016_2019.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117738 entries, 0 to 117737
Data columns (total 82 columns):
Unnamed: 0                        117738 non-null int64
Unnamed: 0.1                      117738 non-null int64
GameId                            117738 non-null int64
GameDate                          117738 non-null object
Quarter                           117738 non-null int64
Minute                            117738 non-null int64
Second                            117738 non-null int64
Remaining                         117738 non-null int64
OffenseTeam                       117738 non-null object
DefenseTeam                       117738 non-null object
Down                              117738 non-null int64
ToGo                              117738 non-null int64
YardLine                          117738 non-null int64
Unnamed: 11                       0 non-null float64
SeriesFirstDown                   117738 non-null int64
Unnamed: 13                       0 non-null floa

<b>Explainations of some variables</b>
- PointDifference = OffenseTeam_CurrentPoint - DefenseTeam_CurrentPoint
- YardLine: Yard position relative to the 0 yard line of the offense side, range [0,99]
- YardLineFixed: Current yard of the offense team, range [0,50]
- YardLineDirection: = 'OWN' if the YardLineFixed is on the offense side, = 'OPP' if it's on the defense side
- Off/Def Pass/Rush indicators: Teams' ranking in different dimensions, 'A' ranks in the first 25%, 'D' ranks in last the 25% among 32 teams. 

In [4]:
df['YardLine-YardLineFixed'] = df['YardLine'] - df['YardLineFixed']
# df['YardLine-YardLineFixed'].unique()
# df['YardLine'].unique()     # range in [0,99], represent yard position relative to the 0 yarf line of the offense
# len(df['YardLineFixed'].unique())       #range in [0,50], use YardLineDirection to point out the position field is on which side

In [ ]:
# df['testYardLine_100'] = df[df['YardLineDirection'] == 'OPP']['YardLineFixed'] + df['YardLine']
# df['testYardLine_100'].unique()      all are 100;  YardLine: yard position relative to the 0 yard line starting from the offense end zone
# df['testYardLine_0'] = df[df['YardLineDirection'] == 'OWN']['YardLineFixed'] - df['YardLine']
# df['testYardLine_0'].unique()     all are 0;  YardLineDirection: current YardLineFixed ([0,50]) is on the offense side or the defense side
# df = df.drop(columns = ['testYardLine_100', 'testYardLine_0'])
# df

In [5]:
df = df.drop(columns = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 11','Unnamed: 13','Unnamed: 17','Unnamed: 18','Challenger','YardLine-YardLineFixed'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117738 entries, 0 to 117737
Data columns (total 75 columns):
GameId                            117738 non-null int64
GameDate                          117738 non-null object
Quarter                           117738 non-null int64
Minute                            117738 non-null int64
Second                            117738 non-null int64
Remaining                         117738 non-null int64
OffenseTeam                       117738 non-null object
DefenseTeam                       117738 non-null object
Down                              117738 non-null int64
ToGo                              117738 non-null int64
YardLine                          117738 non-null int64
SeriesFirstDown                   117738 non-null int64
NextScore                         117738 non-null int64
Description                       117738 non-null object
TeamWin                           117738 non-null int64
SeasonYear                        117738 non-

<b>Summary of Columns

In [6]:
df.columns
# len(df.columns) # = 76

Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'Remaining',
       'OffenseTeam', 'DefenseTeam', 'Down', 'ToGo', 'YardLine',
       'SeriesFirstDown', 'NextScore', 'Description', 'TeamWin', 'SeasonYear',
       'Yards', 'Formation', 'PlayType', 'IsRush', 'IsPass', 'IsIncomplete',
       'IsTouchdown', 'PassType', 'IsSack', 'IsChallenge',
       'IsChallengeReversed', 'IsMeasurement', 'IsInterception', 'IsFumble',
       'IsPenalty', 'IsTwoPointConversion', 'IsTwoPointConversionSuccessful',
       'RushDirection', 'YardLineFixed', 'YardLineDirection',
       'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay', 'PenaltyType',
       'PenaltyYards', 'PointDiff', 'Off Pass Cmp', 'Off Pass Att',
       'Off Pass Cmp%', 'Off Pass TD', 'Off Pass Int%', 'Off Pass Sk%',
       'Off Pass Yds', 'Off Pass Y/G', 'Off Pass Y/A', 'Off Pass QBR',
       'Off Rush Att', 'Off Rush Yds', 'Off Rush Y/G', 'Off Rush Y/A',
       'Off Rush TD', 'Def Pass Cmp', 'Def Pass Att', 'Def Pass Cmp%',
       

<b>Summmary of NULL values

In [7]:
nullvalues = df.loc[:, df.isnull().any()].isnull().sum().sort_values(ascending=False)
print(nullvalues)

PenaltyTeam      109015
PenaltyType      109014
RushDirection     72055
PassType          48523
dtype: int64


In [ ]:
# df.isnull().sum()[20:40] 
# Using the two commands above, we found 'PassType', 'RushDirection', 'PenaltyTeam', 'PenaltyType' have quite many nnull values

<b>Summary of Numerical Values

In [8]:
df[['GameId','GameDate','Quarter','Minute','Second','Remaining','OffenseTeam','DefenseTeam','Down','ToGo','YardLine','SeriesFirstDown']].describe()

,GameId,Quarter,Minute,Second,Remaining,Down,ToGo,YardLine,SeriesFirstDown
count,1.177380e+05,117738.000000,117738.000000,117738.00000,117738.000000,117738.000000,117738.000000,117738.000000,117738.000000
mean,2.017486e+09,2.553483,6.758294,28.58383,1735.946619,1.781362,8.658377,48.244246,0.284420
std,1.059739e+06,1.128155,4.512121,17.75145,1055.274056,0.814531,3.993809,24.408664,0.451139
min,2.016091e+09,1.000000,0.000000,0.00000,-898.000000,0.000000,0.000000,1.000000,0.000000
25%,2.016122e+09,2.000000,3.000000,13.00000,830.250000,1.000000,6.000000,27.000000,0.000000
50%,2.017122e+09,3.000000,7.000000,28.00000,1800.000000,2.000000,10.000000,45.000000,0.000000
75%,2.018121e+09,4.000000,11.000000,44.00000,2629.000000,2.000000,10.000000,67.000000,1.000000
max,2.019112e+09,5.000000,15.000000,59.00000,3600.000000,4.000000,44.000000,99.000000,1.000000


In [9]:
df[['NextScore','Description','TeamWin','SeasonYear','Yards','Formation','PlayType','IsRush','IsPass','IsIncomplete','IsTouchdown','PassType','IsSack','IsChallenge']].describe()

,NextScore,TeamWin,SeasonYear,Yards,IsRush,IsPass,IsIncomplete,IsTouchdown,IsSack,IsChallenge
count,117738.0,117738.0,117738.000000,117738.000000,117738.000000,117738.000000,117738.000000,117738.000000,117738.0,117738.000000
mean,0.0,0.0,2017.364343,6.202135,0.412076,0.587924,0.210629,0.041771,0.0,0.005113
std,0.0,0.0,1.074755,9.006717,0.492211,0.492211,0.407757,0.200066,0.0,0.071323
min,0.0,0.0,2016.000000,-23.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
25%,0.0,0.0,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
50%,0.0,0.0,2017.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.000000
75%,0.0,0.0,2018.000000,9.000000,1.000000,1.000000,0.000000,0.000000,0.0,0.000000
max,0.0,0.0,2019.000000,104.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000


In [10]:
df[['IsChallengeReversed','IsMeasurement','IsInterception','IsFumble','IsPenalty','IsTwoPointConversion']].describe()

,IsChallengeReversed,IsMeasurement,IsInterception,IsFumble,IsPenalty,IsTwoPointConversion
count,117738.000000,117738.0,117738.000000,117738.000000,117738.000000,117738.0
mean,0.002115,0.0,0.014023,0.008995,0.074105,0.0
std,0.045939,0.0,0.117585,0.094413,0.261943,0.0
min,0.000000,0.0,0.000000,0.000000,0.000000,0.0
25%,0.000000,0.0,0.000000,0.000000,0.000000,0.0
50%,0.000000,0.0,0.000000,0.000000,0.000000,0.0
75%,0.000000,0.0,0.000000,0.000000,0.000000,0.0
max,1.000000,0.0,1.000000,1.000000,1.000000,0.0


In [11]:
df[['IsTwoPointConversionSuccessful','RushDirection','YardLineFixed','YardLineDirection','IsPenaltyAccepted','PenaltyTeam','IsNoPlay','PenaltyType','PenaltyYards','PointDiff']].describe()

,IsTwoPointConversionSuccessful,YardLineFixed,IsPenaltyAccepted,IsNoPlay,PenaltyYards,PointDiff
count,117738.0,117738.000000,117738.000000,117738.000000,117738.000000,117738.000000
mean,0.0,29.118033,0.061849,0.046833,0.609684,-1.818470
std,0.0,12.759519,0.240882,0.211281,2.828358,10.482765
min,0.0,1.000000,0.000000,0.000000,0.000000,-49.000000
25%,0.0,20.000000,0.000000,0.000000,0.000000,-7.000000
50%,0.0,30.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,40.000000,0.000000,0.000000,0.000000,4.000000
max,0.0,50.000000,1.000000,1.000000,66.000000,49.000000


In [12]:
df[['RushDirection','YardLineFixed','YardLineDirection','IsPenaltyAccepted','PenaltyTeam','IsNoPlay','PenaltyType','PenaltyYards','PointDiff']].describe()

,YardLineFixed,IsPenaltyAccepted,IsNoPlay,PenaltyYards,PointDiff
count,117738.000000,117738.000000,117738.000000,117738.000000,117738.000000
mean,29.118033,0.061849,0.046833,0.609684,-1.818470
std,12.759519,0.240882,0.211281,2.828358,10.482765
min,1.000000,0.000000,0.000000,0.000000,-49.000000
25%,20.000000,0.000000,0.000000,0.000000,-7.000000
50%,30.000000,0.000000,0.000000,0.000000,0.000000
75%,40.000000,0.000000,0.000000,0.000000,4.000000
max,50.000000,1.000000,1.000000,66.000000,49.000000


- Drop NextScore TeamWin IsSack IsMeasurement IsTwoPointConversion IsTwoPointConversionSuccessful, which are all 0 values

In [13]:
df = df.drop(columns = ['NextScore','TeamWin','IsSack','IsMeasurement','IsTwoPointConversion','IsTwoPointConversionSuccessful'])
df

,GameId,GameDate,Quarter,Minute,Second,Remaining,OffenseTeam,DefenseTeam,Down,ToGo,...,Def Pass Y/A,Def Rush Att,Def Rush Yds,Def Rush Y/G,Def Rush Y/A,Def Rush TD,Off_Pass_Overall,Off_Rush_Overall,Def_Pass_Overall,Def_Rush_Overall
0,2016091102,2016/09/11,3,13,9,1689,HOU,CHI,1,15,...,C,C,B,B,C,C,C,C,B,C
1,2016091105,2016/09/11,2,1,32,1892,NO,OAK,2,10,...,D,B,B,B,D,C,A,C,B,B
2,2016091105,2016/09/11,2,1,11,1871,OAK,NO,1,10,...,D,B,A,A,B,C,A,B,D,B
3,2016091105,2016/09/11,2,1,26,1886,NO,OAK,3,4,...,D,B,B,B,D,C,A,C,B,B
4,2016091105,2016/09/11,3,2,38,1058,NO,OAK,2,1,...,D,B,B,B,D,C,A,C,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117733,2019091600,2019/09/16,3,12,49,1669,CLE,NYJ,1,10,...,B,B,A,A,A,B,C,C,C,A
117734,2019091509,2019/09/15,4,14,37,877,DAL,WAS,3,15,...,C,D,C,C,C,B,A,C,C,C
117735,2019091508,2019/09/15,3,9,18,1458,IND,TEN,3,1,...,B,B,B,B,B,B,C,B,C,B
117736,2019091503,2019/09/15,3,11,35,1595,GB,MIN,1,15,...,B,A,A,A,B,A,B,C,C,A


<b>Summary of Text Values:</b>
- GameDate                          117738 non-null object
- OffenseTeam                       117738 non-null object
- DefenseTeam                       117738 non-null object
- Description                       117738 non-null object
- Formation                         117738 non-null object
- PlayType                          117738 non-null object
- PassType                          69215 non-null object
- RushDirection                     45683 non-null object
- YardLineDirection                 117738 non-null object
- PenaltyTeam                       8723 non-null object
- PenaltyType                       8724 non-null object

In [14]:
df['GameDate'].unique()

array(['2016/09/11', '2016/11/27', '2016/09/18', '2016/09/19',
       '2016/09/22', '2016/09/25', '2016/10/23', '2016/11/28',
       '2016/09/12', '2016/09/26', '2016/09/29', '2016/10/02',
       '2016/09/15', '2016/12/01', '2016/10/16', '2016/10/17',
       '2016/12/04', '2016/10/03', '2016/10/06', '2016/10/09',
       '2016/10/20', '2016/10/24', '2016/10/27', '2016/10/30',
       '2016/10/10', '2016/10/13', '2016/12/11', '2016/10/31',
       '2016/11/06', '2016/11/03', '2017/01/01', '2016/11/07',
       '2016/11/10', '2016/11/13', '2016/11/14', '2016/11/17',
       '2016/11/20', '2016/11/21', '2016/11/24', '2016/12/05',
       '2016/12/08', '2016/12/12', '2016/12/15', '2016/12/17',
       '2016/12/18', '2016/12/19', '2016/12/22', '2016/12/24',
       '2016/12/25', '2016/12/26', '2016/09/08', '2017/09/07',
       '2017/09/10', '2017/09/11', '2017/09/14', '2017/09/17',
       '2017/09/21', '2017/09/24', '2017/09/25', '2017/10/01',
       '2017/10/08', '2017/10/12', '2017/10/15', '2017/

<b>-Team name</b>:
- There're 33 unique values. 'SD' and 'LAC' are the same team, 'Chargers'. We use 'LAC' to represent Chargers.
- To clarify, also replace 'LA' with 'LAR', which represents Los Angeles Rams.

In [18]:
df['OffenseTeam'].unique()    
df['DefenseTeam'].unique()    
# len(df['OffenseTeam'].unique())
# len(df['DefenseTeam'].unique())
# There're 33 unique values. 'SD' and 'LAC' are the same team, 'Chargers'.
# To clarify, also replace 'LA' with 'LAR', which represents Los Angeles Rams

array(['CHI', 'OAK', 'NO', 'NYJ', 'CLE', 'PHI', 'TEN', 'MIN', 'CAR', 'NE',
       'SEA', 'MIA', 'DET', 'ARI', 'SF', 'BAL', 'KC', 'PIT', 'WAS', 'TB',
       'LA', 'DEN', 'IND', 'ATL', 'JAX', 'SD', 'HOU', 'BUF', 'CIN', 'GB',
       'NYG', 'DAL', 'LAC'], dtype=object)

In [19]:
df['OffenseTeam'] = np.where(df['OffenseTeam'] == 'SD','LAC',df['OffenseTeam'])
df['OffenseTeam'] = np.where(df['OffenseTeam'] == 'LA','LAR',df['OffenseTeam'])
print(len(df['OffenseTeam'].unique()))
df['OffenseTeam'].unique()

32


array(['HOU', 'NO', 'OAK', 'CIN', 'PHI', 'CLE', 'MIN', 'TEN', 'CAR', 'NE',
       'NYJ', 'MIA', 'SEA', 'IND', 'SF', 'BAL', 'DET', 'NYG', 'DAL',
       'ARI', 'LAR', 'DEN', 'LAC', 'JAX', 'GB', 'CHI', 'BUF', 'WAS', 'TB',
       'PIT', 'KC', 'ATL'], dtype=object)

In [20]:
df['DefenseTeam'] = np.where(df['DefenseTeam'] == 'SD','LAC',df['DefenseTeam'])
df['DefenseTeam'] = np.where(df['DefenseTeam'] == 'LA','LAR',df['DefenseTeam'])
print(len(df['DefenseTeam'].unique()))
df['DefenseTeam'].unique()

32


array(['CHI', 'OAK', 'NO', 'NYJ', 'CLE', 'PHI', 'TEN', 'MIN', 'CAR', 'NE',
       'SEA', 'MIA', 'DET', 'ARI', 'SF', 'BAL', 'KC', 'PIT', 'WAS', 'TB',
       'LAR', 'DEN', 'IND', 'ATL', 'JAX', 'LAC', 'HOU', 'BUF', 'CIN',
       'GB', 'NYG', 'DAL'], dtype=object)

<b>- Description</b>:
- We found that the column 'Description' briefly describes the pass or rush related players, results and referee final juedgement (if any)

In [21]:
print(df['Description'][5])
print(df['Description'][10000])
print(df['Description'][20000])
print(df['Description'][30000])
print(df['Description'][50000])
print(df['Description'][80000])
print(df['Description'][90000])

(3:18) 30-J.RICHARD RIGHT GUARD TO NO 35 FOR 1 YARD (52-C.ROBERTSON).
(4:32) (SHOTGUN) 12-A.RODGERS PASS SHORT LEFT TO 89-J.COOK TO GB 12 FOR 3 YARDS (26-T.WAYNES).
(7:43) (SHOTGUN) 4-D.CARR PASS SHORT MIDDLE TO 89-A.COOPER TO TB 38 FOR 16 YARDS (38-J.ADJEI-BARIMAH).
(3:43) (NO HUDDLE, SHOTGUN) 10-E.MANNING PASS INCOMPLETE DEEP MIDDLE TO 45-W.TYE (38-A.AMOS) [96-A.HICKS].
(2:00) (SHOTGUN) 25-G.BERNARD LEFT TACKLE TO BAL 47 FOR 4 YARDS (23-T.JEFFERSON). PENALTY ON CIN-62-A.REDMOND, OFFENSIVE HOLDING, 10 YARDS, ENFORCED AT BAL 48.
(12:21) 8-K.COUSINS PASS INCOMPLETE SHORT RIGHT [91-C.WAKE].
(10:16) 14-S.DIGGS LEFT END RAN OB AT PHI 39 FOR 20 YARDS.


<b>- Formation</b>
- Under Center: The quarterback (or another offensive player on occasion) lines up directly behind the center to take the snap. Sometimes referred to as standard formation (as opposed to shotgun formation).
- Wildcat formation: In this formation a running back lines up in the quarterback position and runs the football. Although this formation is pretty much limited to running plays, there is an extra blocker for the runner as the quarterback is not in the backfield

In [22]:
df['Formation'].unique()

array(['SHOTGUN', 'UNDER CENTER', 'NO HUDDLE SHOTGUN', 'NO HUDDLE',
       'FIELD GOAL', 'WILDCAT', 'PUNT'], dtype=object)

In [23]:
df.groupby(['Formation']).size()

Formation
FIELD GOAL               2
NO HUDDLE             2070
NO HUDDLE SHOTGUN     8681
PUNT                     3
SHOTGUN              64765
UNDER CENTER         42212
WILDCAT                  5
dtype: int64

- Only 5 WILDCAT, all of which are rush

In [24]:
df[df['Formation'] == 'WILDCAT'][['ToGo','Formation','Down','IsPass','IsRush','IsIncomplete']]

,ToGo,Formation,Down,IsPass,IsRush,IsIncomplete
17398,10,WILDCAT,1,0,1,0
17411,10,WILDCAT,1,0,1,0
36035,20,WILDCAT,1,0,1,0
69446,10,WILDCAT,1,0,1,0
110117,10,WILDCAT,1,0,1,0


- Only 2 FIELD GOAL, all of which are in the 4th Down and are close to the TouchDown area

In [25]:
df[df['Formation'] == 'FIELD GOAL'][['YardLine','Formation','Down','IsPass','IsRush','IsIncomplete']]

,YardLine,Formation,Down,IsPass,IsRush,IsIncomplete
403,83,FIELD GOAL,4,1,0,1
74392,80,FIELD GOAL,4,1,0,0


- Only 2 kinds of play type, either pass or rush

In [26]:
df['PlayType'].unique()
# df_test = pd.read_csv(r'C:\Users\zhong\Downloads\pbp_2016_2019.csv')
# df_test['PlayType'].unique()

array(['PASS', 'RUSH'], dtype=object)

<b>- PassType:</b>
- Pass are mixed. We only keep the short/deep left/middle/right passtype, that is, 6 passtypes, and replace all the other pass types with NaN values.

In [27]:
df['PassType'].unique()

array(['SHORT LEFT', 'SHORT RIGHT', 'SHORT MIDDLE', 'DEEP LEFT', nan,
       'DEEP RIGHT', 'DEEP MIDDLE', 'NOT LISTED', '(6:33) 11-ASMITH',
       '(4:03) (NO', '(6:41) (SHOTGUN)', 'INTENDED FOR',
       '(55-A.BROOKS) [53-NBOWMAN]', 'MIDDLE TO', '(:15) (SHOTGUN)',
       'KESSLER THROUGH', 'RIGHT (58-JHICKS)',
       '[33-E.GAINES]. LA-33-EGAINES', 'RIGHT. PENALTY',
       '[57-N.SPENCE]. PENALTY', '(94-C.LIUGET) [99-JBOSA]', 'BACK TO',
       'RIGHT TO', 'RULING, AND', '(SHOTGUN) 10-THILL', 'INTERCEPTED BY',
       'MIDDLE. PENALTY', 'LEFT TO'], dtype=object)

In [28]:
df[(df['PassType']=='SHORT LEFT') | (df['PassType']=='DEEP LEFT') \
   | (df['PassType']=='SHORT MIDDLE') | (df['PassType']=='DEEP MIDDLE') \
   | (df['PassType']=='SHORT RIGHT') | (df['PassType']=='DEEP RIGHT')]
# df[(df['PassType'].isnull())]     #  = 48523

,GameId,GameDate,Quarter,Minute,Second,Remaining,OffenseTeam,DefenseTeam,Down,ToGo,...,Def Pass Y/A,Def Rush Att,Def Rush Yds,Def Rush Y/G,Def Rush Y/A,Def Rush TD,Off_Pass_Overall,Off_Rush_Overall,Def_Pass_Overall,Def_Rush_Overall
0,2016091102,2016/09/11,3,13,9,1689,HOU,CHI,1,15,...,C,C,B,B,C,C,C,C,B,C
1,2016091105,2016/09/11,2,1,32,1892,NO,OAK,2,10,...,D,B,B,B,D,C,A,C,B,B
2,2016091105,2016/09/11,2,1,11,1871,OAK,NO,1,10,...,D,B,A,A,B,C,A,B,D,B
3,2016091105,2016/09/11,2,1,26,1886,NO,OAK,3,4,...,D,B,B,B,D,C,A,C,B,B
4,2016091105,2016/09/11,3,2,38,1058,NO,OAK,2,1,...,D,B,B,B,D,C,A,C,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117728,2019092204,2019/09/22,3,3,20,1100,KC,BAL,2,6,...,B,A,A,A,C,B,A,D,B,A
117729,2019092204,2019/09/22,1,0,36,2736,KC,BAL,2,4,...,B,A,A,A,C,B,A,D,B,A
117731,2019091900,2019/09/19,2,13,41,2621,JAX,TEN,2,10,...,B,B,B,B,B,B,B,D,C,B
117733,2019091600,2019/09/16,3,12,49,1669,CLE,NYJ,1,10,...,B,B,A,A,A,B,C,C,C,A


In [29]:
df['PassType'] = np.where((df['PassType']!='SHORT LEFT') & (df['PassType']!='DEEP LEFT') \
   & (df['PassType']!='SHORT MIDDLE') & (df['PassType']!='DEEP MIDDLE') \
   & (df['PassType']!='SHORT RIGHT') & (df['PassType']!='DEEP RIGHT'),np.NaN,df['PassType'])

In [30]:
print(len(df['PassType'].unique()))
df['PassType'].unique()

7


array(['SHORT LEFT', 'SHORT RIGHT', 'SHORT MIDDLE', 'DEEP LEFT', nan,
       'DEEP RIGHT', 'DEEP MIDDLE'], dtype=object)

<b>- RushDirection:</b>

In [31]:
df['RushDirection'].unique()

array([nan, 'RIGHT GUARD', 'CENTER', 'LEFT END', 'RIGHT TACKLE',
       'LEFT TACKLE', 'RIGHT END', 'LEFT GUARD'], dtype=object)

<b>- YardLine Direction:</b>
- whether the play is in own half filed, or opponent's half field

In [32]:
df['YardLineDirection'].unique()
# whether the play is in own half filed, or opponent's half field

array(['OPP', 'OWN'], dtype=object)

<b>- PenaltyTeam:</b>

In [33]:
df['PenaltyTeam'].unique()
df['PenaltyTeam'] = np.where(df['PenaltyTeam'] == 'SD','LAC',df['PenaltyTeam'])
df['PenaltyTeam'] = np.where(df['PenaltyTeam'] == 'LA','LAR',df['PenaltyTeam'])
print(len(df['PenaltyTeam'].unique()))
df['PenaltyTeam'].unique()

33


array([nan, 'OAK', 'SEA', 'CLE', 'TEN', 'NE', 'PHI', 'HOU', 'DEN', 'CIN',
       'DET', 'WAS', 'IND', 'KC', 'LAR', 'BAL', 'NO', 'MIN', 'SF', 'ARI',
       'DAL', 'GB', 'LAC', 'NYJ', 'JAX', 'MIA', 'PIT', 'ATL', 'BUF', 'TB',
       'CAR', 'NYG', 'CHI'], dtype=object)

<b>Generate team offense & defense ranking 1,2,3,4 numeric values</b>
- For further analysis, we replace A,B,C,D with 1,2,3,4

In [34]:
df

,GameId,GameDate,Quarter,Minute,Second,Remaining,OffenseTeam,DefenseTeam,Down,ToGo,...,Def Pass Y/A,Def Rush Att,Def Rush Yds,Def Rush Y/G,Def Rush Y/A,Def Rush TD,Off_Pass_Overall,Off_Rush_Overall,Def_Pass_Overall,Def_Rush_Overall
0,2016091102,2016/09/11,3,13,9,1689,HOU,CHI,1,15,...,C,C,B,B,C,C,C,C,B,C
1,2016091105,2016/09/11,2,1,32,1892,NO,OAK,2,10,...,D,B,B,B,D,C,A,C,B,B
2,2016091105,2016/09/11,2,1,11,1871,OAK,NO,1,10,...,D,B,A,A,B,C,A,B,D,B
3,2016091105,2016/09/11,2,1,26,1886,NO,OAK,3,4,...,D,B,B,B,D,C,A,C,B,B
4,2016091105,2016/09/11,3,2,38,1058,NO,OAK,2,1,...,D,B,B,B,D,C,A,C,B,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117733,2019091600,2019/09/16,3,12,49,1669,CLE,NYJ,1,10,...,B,B,A,A,A,B,C,C,C,A
117734,2019091509,2019/09/15,4,14,37,877,DAL,WAS,3,15,...,C,D,C,C,C,B,A,C,C,C
117735,2019091508,2019/09/15,3,9,18,1458,IND,TEN,3,1,...,B,B,B,B,B,B,C,B,C,B
117736,2019091503,2019/09/15,3,11,35,1595,GB,MIN,1,15,...,B,A,A,A,B,A,B,C,C,A


In [35]:
print(len(df.columns))
df.columns

69


Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'Remaining',
       'OffenseTeam', 'DefenseTeam', 'Down', 'ToGo', 'YardLine',
       'SeriesFirstDown', 'Description', 'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsChallenge', 'IsChallengeReversed', 'IsInterception',
       'IsFumble', 'IsPenalty', 'RushDirection', 'YardLineFixed',
       'YardLineDirection', 'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay',
       'PenaltyType', 'PenaltyYards', 'PointDiff', 'Off Pass Cmp',
       'Off Pass Att', 'Off Pass Cmp%', 'Off Pass TD', 'Off Pass Int%',
       'Off Pass Sk%', 'Off Pass Yds', 'Off Pass Y/G', 'Off Pass Y/A',
       'Off Pass QBR', 'Off Rush Att', 'Off Rush Yds', 'Off Rush Y/G',
       'Off Rush Y/A', 'Off Rush TD', 'Def Pass Cmp', 'Def Pass Att',
       'Def Pass Cmp%', 'Def Pass TD', 'Def Pass Int%', 'Def Pass Sk%',
       'Def Pass Yds', 'Def Pass Y/G', 'Def Pass Y/A', 'Def Rush Att',
    

In [36]:
df = df.rename(columns = { 'Off Pass Cmp':'Off_Pass_Cmp','Off Pass Att':'Off_Pass_Att','Off Pass Cmp%':'Off_Pass_Cmp_pct',\
                          'Off Pass TD':'Off_Pass_TD','Off Pass Int%':'Off_Pass_Int_pct','Off Pass Sk%':'Off_Pass_Sk_pct' ,\
                          'Off Pass Yds':'Off_Pass_Yds','Off Pass Y/G':'Off_Pass_YperG','Off Pass Y/A':'Off_Pass_YperA',\
                          'Off Pass QBR':'Off_Pass_QBR','Off Rush Att':'Off_Rush_Att','Off Rush Yds':'Off_Rush_Yds',\
                          'Off Rush Y/G':'Off_Rush_YperG','Off Rush Y/A':'Off_Rush_YperA','Off Rush TD':'Off_Rush_TD',\
                          'Def Pass Cmp': 'Def_Pass_Cmp','Def Pass Att':'Def_Pass_Att','Def Pass Cmp%': 'Def_Pass_Cmp_pct',\
                          'Def Pass TD':'Def_Pass_TD','Def Pass Int%':'Def_Pass_Int_pct','Def Pass Sk%':'Def_Pass_Sk_pct',\
                          'Def Pass Yds':'Def_Pass_Yds','Def Pass Y/G':'Def_Pass_YperG','Def Pass Y/A':'Def_Pass_YperA',\
                          'Def Rush Att':'Def_Rush_Att','Def Rush Yds':'Def_Rush_Yds','Def Rush Y/G':'Def_Rush_YperG',\
                          'Def Rush Y/A':'Def_Rush_YperA','Def Rush TD':'Def_Rush_TD'})

In [37]:
print(len(df.columns))
df.columns

69


Index(['GameId', 'GameDate', 'Quarter', 'Minute', 'Second', 'Remaining',
       'OffenseTeam', 'DefenseTeam', 'Down', 'ToGo', 'YardLine',
       'SeriesFirstDown', 'Description', 'SeasonYear', 'Yards', 'Formation',
       'PlayType', 'IsRush', 'IsPass', 'IsIncomplete', 'IsTouchdown',
       'PassType', 'IsChallenge', 'IsChallengeReversed', 'IsInterception',
       'IsFumble', 'IsPenalty', 'RushDirection', 'YardLineFixed',
       'YardLineDirection', 'IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay',
       'PenaltyType', 'PenaltyYards', 'PointDiff', 'Off_Pass_Cmp',
       'Off_Pass_Att', 'Off_Pass_Cmp_pct', 'Off_Pass_TD', 'Off_Pass_Int_pct',
       'Off_Pass_Sk_pct', 'Off_Pass_Yds', 'Off_Pass_YperG', 'Off_Pass_YperA',
       'Off_Pass_QBR', 'Off_Rush_Att', 'Off_Rush_Yds', 'Off_Rush_YperG',
       'Off_Rush_YperA', 'Off_Rush_TD', 'Def_Pass_Cmp', 'Def_Pass_Att',
       'Def_Pass_Cmp_pct', 'Def_Pass_TD', 'Def_Pass_Int_pct',
       'Def_Pass_Sk_pct', 'Def_Pass_Yds', 'Def_Pass_YperG', 'Def_Pas

<b>Generate team offense & defense ranking variables</b>
- For readability, we add '_rank' to the end of each variable name

In [38]:
df['Off_Pass_Cmp_rank'] = 0
df['Off_Pass_Att_rank'] = 0
df['Off_Pass_Cmp%_rank'] = 0
df['Off_Pass_TD_rank'] = 0
df['Off_Pass_Int%_rank'] = 0
df['Off_Pass_Sk%_rank'] = 0 
df['Off_Pass_Yds_rank'] = 0
df['Off_Pass_Y/G_rank'] = 0
df['Off_Pass_Y/A_rank'] = 0
df['Off_Pass_QBR_rank'] = 0
df['Off_Rush_Att_rank'] = 0
df['Off_Rush_Yds_rank'] = 0
df['Off_Rush_Y/G_rank'] = 0
df['Off_Rush_Y/A_rank'] = 0
df['Off_Rush_TD_rank'] = 0 
df['Def_Pass_Cmp_rank'] = 0 
df['Def_Pass_Att_rank'] = 0
df['Def_Pass_Cmp%_rank'] = 0 
df['Def_Pass_TD_rank'] = 0
df['Def_Pass_Int%_rank'] = 0
df['Def_Pass_Sk%_rank'] = 0
df['Def_Pass_Yds_rank'] = 0
df['Def_Pass_Y/G_rank'] = 0
df['Def_Pass_Y/A_rank'] = 0
df['Def_Rush_Att_rank'] = 0
df['Def_Rush_Yds_rank'] = 0
df['Def_Rush_Y/G_rank'] = 0
df['Def_Rush_Y/A_rank'] = 0
df['Def_Rush_TD_rank'] = 0
df['Off_Pass_Overall_rank'] = 0
df['Off_Rush_Overall_rank'] = 0
df['Def_Pass_Overall_rank'] = 0
df['Def_Rush_Overall_rank'] = 0

In [39]:
def gen_rank(column):
    # Offense
    if column == 'A':
        return 1
    if column == 'B':
        return 2
    if column == 'C':
        return 3
    if column == 'D':
        return 4

In [40]:
df['Off_Pass_Cmp_rank'] = df.Off_Pass_Cmp.apply(lambda x : gen_rank(x))
df['Off_Pass_Att_rank'] = df.Off_Pass_Att.apply(lambda x : gen_rank(x))
df['Off_Pass_Cmp%_rank'] = df.Off_Pass_Cmp_pct.apply(lambda x : gen_rank(x))
df['Off_Pass_TD_rank'] = df.Off_Pass_TD.apply(lambda x : gen_rank(x))
df['Off_Pass_Int%_rank'] = df.Off_Pass_Int_pct.apply(lambda x : gen_rank(x))
df['Off_Pass_Sk%_rank'] =  df.Off_Pass_Sk_pct.apply(lambda x : gen_rank(x))
df['Off_Pass_Yds_rank'] = df.Off_Pass_Yds.apply(lambda x : gen_rank(x))
df['Off_Pass_Y/G_rank'] = df.Off_Pass_YperG.apply(lambda x : gen_rank(x))
df['Off_Pass_Y/A_rank'] = df.Off_Pass_YperA.apply(lambda x : gen_rank(x))
df['Off_Pass_QBR_rank'] = df.Off_Pass_QBR.apply(lambda x : gen_rank(x))

df['Off_Rush_Att_rank'] = df.Off_Rush_Att.apply(lambda x : gen_rank(x))
df['Off_Rush_Yds_rank'] = df.Off_Rush_Yds.apply(lambda x : gen_rank(x))
df['Off_Rush_Y/G_rank'] = df.Off_Rush_YperG.apply(lambda x : gen_rank(x))
df['Off_Rush_Y/A_rank'] = df.Off_Rush_YperA.apply(lambda x : gen_rank(x))
df['Off_Rush_TD_rank'] = df.Off_Rush_TD.apply(lambda x : gen_rank(x))

In [41]:
df['Def_Pass_Cmp_rank'] = df.Def_Pass_Cmp.apply(lambda x : gen_rank(x))
df['Def_Pass_Att_rank'] = df.Def_Pass_Att.apply(lambda x : gen_rank(x))
df['Def_Pass_Cmp%_rank'] = df.Def_Pass_Cmp_pct.apply(lambda x : gen_rank(x))
df['Def_Pass_TD_rank'] = df.Def_Pass_TD.apply(lambda x : gen_rank(x))
df['Def_Pass_Int%_rank'] = df.Def_Pass_Int_pct.apply(lambda x : gen_rank(x))
df['Def_Pass_Sk%_rank'] = df.Def_Pass_Sk_pct.apply(lambda x : gen_rank(x))
df['Def_Pass_Yds_rank'] = df.Def_Pass_Yds.apply(lambda x : gen_rank(x))
df['Def_Pass_Y/G_rank'] = df.Def_Pass_YperG.apply(lambda x : gen_rank(x))
df['Def_Pass_Y/A_rank'] = df.Def_Pass_YperA.apply(lambda x : gen_rank(x))

df['Def_Rush_Att_rank'] = df.Def_Rush_Att.apply(lambda x : gen_rank(x))
df['Def_Rush_Yds_rank'] = df.Def_Rush_Yds.apply(lambda x : gen_rank(x))
df['Def_Rush_Y/G_rank'] = df.Def_Rush_YperG.apply(lambda x : gen_rank(x))
df['Def_Rush_Y/A_rank'] = df.Def_Rush_YperA.apply(lambda x : gen_rank(x))
df['Def_Rush_TD_rank'] = df.Def_Rush_TD.apply(lambda x : gen_rank(x))

df['Off_Pass_Overall_rank'] = df.Off_Pass_Overall.apply(lambda x : gen_rank(x))
df['Off_Rush_Overall_rank'] = df.Off_Rush_Overall.apply(lambda x : gen_rank(x))
df['Def_Pass_Overall_rank'] = df.Def_Pass_Overall.apply(lambda x : gen_rank(x))
df['Def_Rush_Overall_rank'] = df.Def_Rush_Overall.apply(lambda x : gen_rank(x))

In [42]:
df

,GameId,GameDate,Quarter,Minute,Second,Remaining,OffenseTeam,DefenseTeam,Down,ToGo,...,Def_Pass_Y/A_rank,Def_Rush_Att_rank,Def_Rush_Yds_rank,Def_Rush_Y/G_rank,Def_Rush_Y/A_rank,Def_Rush_TD_rank,Off_Pass_Overall_rank,Off_Rush_Overall_rank,Def_Pass_Overall_rank,Def_Rush_Overall_rank
0,2016091102,2016/09/11,3,13,9,1689,HOU,CHI,1,15,...,3,3,2,2,3,3,3,3,2,3
1,2016091105,2016/09/11,2,1,32,1892,NO,OAK,2,10,...,4,2,2,2,4,3,1,3,2,2
2,2016091105,2016/09/11,2,1,11,1871,OAK,NO,1,10,...,4,2,1,1,2,3,1,2,4,2
3,2016091105,2016/09/11,2,1,26,1886,NO,OAK,3,4,...,4,2,2,2,4,3,1,3,2,2
4,2016091105,2016/09/11,3,2,38,1058,NO,OAK,2,1,...,4,2,2,2,4,3,1,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117733,2019091600,2019/09/16,3,12,49,1669,CLE,NYJ,1,10,...,2,2,1,1,1,2,3,3,3,1
117734,2019091509,2019/09/15,4,14,37,877,DAL,WAS,3,15,...,3,4,3,3,3,2,1,3,3,3
117735,2019091508,2019/09/15,3,9,18,1458,IND,TEN,3,1,...,2,2,2,2,2,2,3,2,3,2
117736,2019091503,2019/09/15,3,11,35,1595,GB,MIN,1,15,...,2,1,1,1,2,1,2,3,3,1


<b>Choose 2 variables to check if the generator is correct</b>

In [43]:
df[['Off_Pass_Sk_pct', 'Off_Pass_Sk%_rank']][:30]

,Off_Pass_Sk_pct,Off_Pass_Sk%_rank
0,B,2
1,A,1
2,A,1
3,A,1
4,A,1
5,A,1
6,C,3
7,B,2
8,B,2
9,B,2


In [44]:
df[['Def_Rush_Att','Def_Rush_Att_rank']][500:530]

,Def_Rush_Att,Def_Rush_Att_rank
500,B,2
501,A,1
502,B,2
503,B,2
504,B,2
505,B,2
506,B,2
507,A,1
508,A,1
509,A,1


In [ ]:
# df = df.drop(columns = ['SeriesFirstDown', 'PlayType','IsIncomplete', 'IsTouchdown', 'IsChallenge', 'IsChallengeReversed',\
#                                     'IsInterception','IsFumble', 'IsPenalty','IsPenaltyAccepted', 'PenaltyTeam', 'IsNoPlay','PenaltyType', \
#                                     'PenaltyYards'])

# df = df.drop(columns = ['Off_Pass_Cmp', 'Off_Pass_Att','Off_Pass_Cmp_pct', 'Off_Pass_TD', 'Off_Pass_Int_pct','Off_Pass_Sk_pct',\
#                          'Off_Pass_Yds', 'Off_Pass_YperG', 'Off_Pass_YperA', 'Off_Pass_QBR', 'Off_Rush_Att', 'Off_Rush_Yds', \
#                          'Off_Rush_YperG', 'Off_Rush_YperA', 'Off_Rush_TD', 'Def_Pass_Cmp', 'Def_Pass_Att', 'Def_Pass_Cmp_pct', \
#                          'Def_Pass_TD', 'Def_Pass_Int_pct', 'Def_Pass_Sk_pct', 'Def_Pass_Yds', 'Def_Pass_YperG', 'Def_Pass_YperA',\
#                          'Def_Rush_Att', 'Def_Rush_Yds', 'Def_Rush_YperG', 'Def_Rush_YperA','Def_Rush_TD', 'Off_Pass_Overall', \
#                          'Off_Rush_Overall','Def_Pass_Overall', 'Def_Rush_Overall'])

# df = df.drop(columns = ['Off_Pass_Cmp%_rank','Def_Pass_Cmp%_rank','Off_Pass_Yds_rank','Def_Pass_Yds_rank','Off_Rush_Y/G_rank',\
#                                     'Def_Rush_Y/G_rank'])

# df.shape

# Export to csv file

In [45]:
df.to_csv(r'C:\Users\zhong\Downloads\pbp_2016_2019_cleaned_data_updated.csv', index = False)

In [46]:
df

,GameId,GameDate,Quarter,Minute,Second,Remaining,OffenseTeam,DefenseTeam,Down,ToGo,...,Def_Pass_Y/A_rank,Def_Rush_Att_rank,Def_Rush_Yds_rank,Def_Rush_Y/G_rank,Def_Rush_Y/A_rank,Def_Rush_TD_rank,Off_Pass_Overall_rank,Off_Rush_Overall_rank,Def_Pass_Overall_rank,Def_Rush_Overall_rank
0,2016091102,2016/09/11,3,13,9,1689,HOU,CHI,1,15,...,3,3,2,2,3,3,3,3,2,3
1,2016091105,2016/09/11,2,1,32,1892,NO,OAK,2,10,...,4,2,2,2,4,3,1,3,2,2
2,2016091105,2016/09/11,2,1,11,1871,OAK,NO,1,10,...,4,2,1,1,2,3,1,2,4,2
3,2016091105,2016/09/11,2,1,26,1886,NO,OAK,3,4,...,4,2,2,2,4,3,1,3,2,2
4,2016091105,2016/09/11,3,2,38,1058,NO,OAK,2,1,...,4,2,2,2,4,3,1,3,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117733,2019091600,2019/09/16,3,12,49,1669,CLE,NYJ,1,10,...,2,2,1,1,1,2,3,3,3,1
117734,2019091509,2019/09/15,4,14,37,877,DAL,WAS,3,15,...,3,4,3,3,3,2,1,3,3,3
117735,2019091508,2019/09/15,3,9,18,1458,IND,TEN,3,1,...,2,2,2,2,2,2,3,2,3,2
117736,2019091503,2019/09/15,3,11,35,1595,GB,MIN,1,15,...,2,1,1,1,2,1,2,3,3,1
